<a href="https://colab.research.google.com/github/dark-matter08/scrappers/blob/main/trekbike_site_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Website: https://www.trekbikes.com/us/en_US/bikes/c/B100/

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
}

In [ ]:
default_url = "https://www.trekbikes.com"
links = []
page_num = 0
while page_num != 2:
  response = requests.get(f"https://www.trekbikes.com/us/en_US/bikes/c/B100/?q=%3Arelevance&page={page_num}&pageSize=24", headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  bikes = soup.find_all("li", class_="cell productListItem product-list__item")
  for bike in bikes:
    try:
      link = (bike.find("a", {"class":"product-tile__link"}))
      link = link.get('href')
      link = default_url + link
      links.append(link)
    except Exception as e:
      print(f"======{e}======")
  page_num = page_num + 1 
  print(f"============================================={page_num}=======================================================") 

print(links)
print(len(links))

======'NoneType' object has no attribute 'get'======
=============================================1=======================================================
======'NoneType' object has no attribute 'get'======
=============================================2=======================================================
['https://www.trekbikes.com/us/en_US/bikes/mountain-bikes/electric-mountain-bikes/fuel-exe/fuel-exe-9-9-xx1-axs/p/36373/', 'https://www.trekbikes.com/us/en_US/bikes/road-bikes/performance-road-bikes/domane/domane-slr/domane-slr-9-etap-gen-4/p/37358/', 'https://www.trekbikes.com/us/en_US/bikes/road-bikes/performance-road-bikes/madone/madone-slr/madone-slr-9-etap-gen-7/p/37420/', 'https://www.trekbikes.com/us/en_US/bikes/road-bikes/performance-road-bikes/%C3%A9monda/%C3%A9monda-slr/%C3%A9monda-slr-9-etap/p/33140/', 'https://www.trekbikes.com/us/en_US/bikes/road-bikes/performance-road-bikes/domane/domane-slr/domane-slr-9/p/36742/', 'https://www.trekbikes.com/us/en_US/bikes/road-bikes/

In [ ]:
names = []
categories = []
prices = []
short_description_list = []
main_image_list = []
main_video_list = []
overview_section_list = []
features_section_list = []
specs_section_list = []
count = 1

for link in links[:1]:
  print(f"================================== Link Count: {count}================================================")
  print(link)
  link_response = requests.get(link, headers=headers)
  link_soup = BeautifulSoup(link_response.content, 'html.parser')

  first_div = link_soup.find("div", {"class":"fcp-reveal"})


  # ====================================================== Actual Data ==========================================================
  name = link_soup.find('li', {'class': 'text--bold'}).text
  price = link_soup.find('input', {'id': 'gtm-product-display-price'}).get('value')
  short_description = link_soup.find('pdp-buying-zone-container').get("copy-positioning-statement")
  main_image = link_soup.find('meta', {"property" : "og:image" }).get('content')

  try:
    main_video = link_soup.find('meta', {"property" : "og:video" }).get('content')
  except Exception as e:
    main_video = ''

  try:
    category = link.split('/')[6]
    category = category.split('-')
    category = ' '.join(category).capitalize()
    category = " ".join(word[0].upper()+word[1:] for word in category.split(" "))

  except Exception as e:
    category = ''

  try:
    overview_section = link_soup.find_all('section')[3].find_all('div')
    overview_section = [str(section_single) for section_single in overview_section]
    overview_section = ' '.join(overview_section)
    overview_section = overview_section.replace('data-src', 'src')
    overview_section = overview_section.replace('<i class="md-18 mr-1">favorite_border</i>', '')
    overview_section = overview_section.replace('class="vid-thumbnail thumb lazyload"', 'style="width: 500px; height: 300; margin-right: auto; margin-left: auto;"')
    
  except Exception as e:
    overview_section = ''

  try:
    features_section = link_soup.find('section', {'class': 'productPrimaryFeaturesComponent'})
    features_section = str(features_section)
    features_section = features_section.replace('data-src', 'src')
    features_section = features_section.replace('class="primary-feature__card sprocket__card"', 'style="display: flex; flex-direction: row; margin-top: 20px"')
    features_section = features_section.replace('class="primary-feature__card__image sprocket__card__image"', 'style="width: 500px; margin-right: 10px"')
    features_section = features_section.replace('class="w-full lazyload"', 'style="width: 350px; height: 190px;"')
    
  except Exception as e:
    features_section = ''

  try:
    specs_section = link_soup.find_all('table')
    specs_section = [str(section_single) for section_single in specs_section]
    specs_section = ' '.join(specs_section)
    specs_section = specs_section.replace('data-src', 'src')
    specs_section = specs_section.replace('class="sprocket__table spec"', 'style="margin-top: 20px; width: 100%; border:1px solid black;"')
    specs_section = specs_section.replace('style="vertical-align: text-bottom;"', 'style="vertical-align: text-bottom; font-weight: 600; border:1px solid black; padding: 10px"')
    specs_section = specs_section.replace('<td', '<td style="border:1px solid black; padding: 10px"')
    specs_section = specs_section.replace('<th', '<th style="border:1px solid black; padding: 10px"')

  except Exception as e:
    specs_section = ''


  # try:
  #   geometry_image = link_soup.find_all('template')[19]
  #   geometry_table = link_soup.find_all('template')[20]
  #   geometry_section = f'<div style="display: flex; flex-direction: row">{geometry_image} {geometry_table}</div>'
  #   geometry_section = geometry_section.replace('<template #geometry="">', '')
  #   geometry_section = geometry_section.replace('</template> ', '')
  #   geometry_section = geometry_section.replace('data-src', 'src')
  #   geometry_section = geometry_section.replace('template', 'table')
  #   geometry_section = geometry_section.replace('class="sizing-table__bike-image"', 'style="width: 300px"')
  #   geometry_section = geometry_section.replace('class=" lazyload"', 'style="width: 350px; height: 190px"')
  #   print((geometry_section))

  # except Exception as e:
  #   geometry_section = ''

  # ============================================================ Adding Data to Lists =======================================================
  names.append(name)
  categories.append(category)
  prices.append(price)
  short_description_list.append(short_description)
  main_image_list.append(main_image)
  main_video_list.append(main_video)
  overview_section_list.append(overview_section)
  features_section_list.append(features_section)
  specs_section_list.append(specs_section)
  print("==========================================================================================================================================")
  count = count + 1


================================== Link Count: 1================================================
https://www.trekbikes.com/us/en_US/bikes/mountain-bikes/electric-mountain-bikes/fuel-exe/fuel-exe-9-9-xx1-axs/p/36373/


In [ ]:
data = {"Name": names, "Category": categories, "Price": prices, "Short Description": short_description_list, "Main Image": main_image_list, "Main Video": main_video_list, "Overview": overview_section_list,"Features": features_section_list, "Specs": specs_section_list}
df = pd.DataFrame(data=data)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/trekbikes.csv')
df